# Assignment-2 Part 2 - Automata Machine
###Description
Given a randomly generated set of rules, an 8-bit initial state, and an 8-bit goal state, write a GA that finds the set of rules that will transform the initial state to the goal state after some amount of passes.

###Instructions
An individual is a set of randomly generated rules based off of a 5-bit truth table. 

Example:

Input | Rule
--- | ---
00000 | 1
00001 | 0
00010 | 3
...   | ...
11110 | 2
11111 | 2

The rules work as follows:


> 0 - replace middle value with a 0

> 1 - replace middle value with a 1

> 2 - delete the middle value

> 3 - replicate the middle value

You will use a 5-bit sliding window to implement the rules on the initial state. It may dramatically shorten run time if you parallelize the sliding window.

###Example: 5-bit Initial State and 3-bit Sliding Window###
Input | Rule
--- | ---
000 | 0
001 | 1
010 | 2
011 | 3
100 | 3
101 | 2
110 | 1
111 | 0

Initial State: 01000

Goal State: 11111

Current State = copy(Initial State)

Current State: 01000

**First Pass**

Next State = copy(Current State)

Next State: 01000

Step | Current State | Sliding Window | Rule | Change to Next State this Pass
--- | --- | --- | --- | ---
0 | 01000 |  |  |  01000 (note this is a copy of Current State)
1 | ***010***00 | 010 | 2 |  0***_***000
2 | 0***100***0 | 100 | 3 | 00***0***00
3 | 01***000*** | 000 | 0 | 000***0***0
4 | ***0***10***00*** | 000 | 0 | 0000***0***
5 | ***01***00***0*** | 001 | 1 | ***1***0000

**During each pass ONLY change Next State - DO NOT CHANGE THE INITIAL/CURRENT STATE**

Update Current State once all passes are completed.

Next State: 10000

Current State = copy(Next State)

Current State: 10000

**End of First Pass**



It may take multiple passes to get to the goal state. Some rules will you get close to the goal state, but never fully reach it.

Population is a JSON file with the desired number(population_size) of individuals. The first and second elements refer to the intial state and goal state respectively.

Running the intialize_population function, generates the JSON file - automata-population.json.

# Supporting Codes
Below are the supporting codes necessary to generate the initial state, goal state, and initial population

**Whenever you open the notebook just run the below codes once**.

After running them once all the functions can be reused anywhere in the notebook.

In [0]:
## Importing all the packages
import numpy as np
import random
import json

In [0]:
################################################################################
#
#   Initializes the population of candidates with a size of population_size. 
#   Also initializes your 8-bit initial_state and your 8-bit goal_state.
#
#   Parameters:
#   population_size - an integer defining how many rules tables to generate
#
################################################################################
def initialize_population(population_size=3):
    new_population = []
    random_truth_table = []

    initial_state = format(np.random.randint(256), 'b')
    goal_state = format(np.random.randint(256), 'b')

    while len(initial_state) != 8:
      initial_state = "0" + initial_state

    while len(goal_state) != 8:
      goal_state = "0" + goal_state

    while np.array_equal(initial_state, goal_state):
      goal_state = np.random.randint(2,size=(8,))

    new_population.append({"Initial State": str(initial_state)})
    new_population.append({"Goal State": str(goal_state)})

    for individual in range(population_size):
      random_truth_table= list()
      for bit in range(32):
        key = format(bit, 'b')
        while len(key) != 5:
          key = "0" + key
        random_truth_table.append({key: np.random.randint(4)})
      new_population.append(random_truth_table)
    
    with open("automata-population.json", 'w') as o:
      o.write(json.dumps(new_population))

## Generating Initial Population

After running all the codes above once, Use the function **initialize_population(population_size)** to generate a new initial state, goal state, and set of rules tables.

One JSON file is generated: automata-population.json.

Download this file and use it as need.

In [0]:
initialize_population(population_size = 5)

with open("automata-population.json",'r') as f:
  population = json.loads(f.read());

initial_state = population[0]['Initial State']
goal_state = population[1]['Goal State']

current_state = initial_state

# Visualizing a Population

To visualize your population, use the function print_population().

If you have modified the population while running your GA, make sure it matches the same format as the initial_population. Refer to initialize_population for the format.


In [0]:
################################################################################
#
#   Prints the passed in population population
#
#   Parameters:
#   population_json - json object containing the population to print
#
################################################################################
def print_population(population_json):
  json_obj = json.loads(population_json)

  for individual in range(len(population)):
    if individual < 2:
      print(str(population[individual]))
    else:
      print(population[individual])
      #print("Invididual " + str(individual - 1) + ": " + str(population[individual]))

In [0]:
print_population(json.dumps(population))

{'Initial State': '11000101'}
{'Goal State': '11011101'}
[{'00000': 0}, {'00001': 3}, {'00010': 0}, {'00011': 2}, {'00100': 1}, {'00101': 2}, {'00110': 3}, {'00111': 2}, {'01000': 3}, {'01001': 0}, {'01010': 3}, {'01011': 2}, {'01100': 0}, {'01101': 0}, {'01110': 0}, {'01111': 0}, {'10000': 0}, {'10001': 1}, {'10010': 0}, {'10011': 1}, {'10100': 2}, {'10101': 2}, {'10110': 3}, {'10111': 1}, {'11000': 1}, {'11001': 2}, {'11010': 1}, {'11011': 2}, {'11100': 0}, {'11101': 0}, {'11110': 1}, {'11111': 2}]
[{'00000': 1}, {'00001': 0}, {'00010': 3}, {'00011': 0}, {'00100': 1}, {'00101': 1}, {'00110': 1}, {'00111': 2}, {'01000': 0}, {'01001': 0}, {'01010': 2}, {'01011': 0}, {'01100': 0}, {'01101': 0}, {'01110': 3}, {'01111': 0}, {'10000': 2}, {'10001': 1}, {'10010': 3}, {'10011': 2}, {'10100': 0}, {'10101': 1}, {'10110': 2}, {'10111': 3}, {'11000': 0}, {'11001': 3}, {'11010': 3}, {'11011': 1}, {'11100': 0}, {'11101': 2}, {'11110': 0}, {'11111': 3}]
[{'00000': 0}, {'00001': 0}, {'00010': 1}, {'

# Fitness Evaluation

Fitness of an individual is calculated using Minimum Edit Distance (MED). The more deletes, inserts, and substitutions required to convert the final state to the goal state, the worse the fitness. A lower fitness value is better.

A sample of the fitness evaluation(written in python) is shared, please refer to the function calculate_fitness().

If you want to understand how this fitness function works, please refer to:
[Minimum Edit Distance (MED)](https://archive.org/details/31DefiningMinimumEditDistanceStanfordNLPProfessorDanJurafskyChrisManning/3+-+2+-+Computing+Minimum+Edit+Distance+-+Stanford+NLP+-+Professor+Dan+Jurafsky+%26+Chris+Manning.mp4)

In [0]:
################################################################################
#
#   Calculates the Minimum Edit Distance (MED) between two strings using dynamic
#   programming. The lower the value, the better.
#
#   Parameters:
#   final_state - the result of applying your GA strategy on the initial_state
#   goal_state - the state that you are aiming to change the initial_state into
#
#   Returns:
#   an integer corresponding to the fitness of your final_state vs goal_state
#
################################################################################
def calculate_fitness(final_state, goal_state):
  fitness_table = np.zeros((len(final_state) + 1 , len(goal_state) + 1), dtype=int)

  for row in range(len(final_state)+1):
    fitness_table[row][0] = row
  for col in range(len(goal_state)+1):
    fitness_table[0][col] = col

  for row in range(len(final_state)+1):
    for col in range(len(goal_state)+1):
      if row != 0 and col != 0:  
        if final_state[row-1] == goal_state[col-1]:
          fitness_table[row][col] = fitness_table[row-1][col-1]
        else:
            fitness_table[row][col] = min(fitness_table[row-1][col] + 1,
                                          fitness_table[row][col-1] + 1,
                                          fitness_table[row-1][col-1] + 2
            )
  return fitness_table[len(final_state)][len(goal_state)]

In [0]:
calculate_fitness(current_state, goal_state)

4